In [2]:
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE_URL = "https://thedyrt.com"

def get_all_states_urls() -> list[str]:
    resp = requests.get(BASE_URL)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "lxml")

    pattern = re.compile(r"^/camping/[a-z-]+$")
    urls = []
    for a in soup.find_all("a", href=pattern):
        full = urljoin(BASE_URL, a["href"])
        urls.append(full)

    return list(dict.fromkeys(urls))

if __name__ == "__main__":
    states = get_all_states_urls()
    print(f"[✓] {len(states)} state URL found:\n")
    for u in states:
        print(u)
        


[✓] 52 state URL found:

https://thedyrt.com/camping/alabama
https://thedyrt.com/camping/alaska
https://thedyrt.com/camping/arizona
https://thedyrt.com/camping/arkansas
https://thedyrt.com/camping/california
https://thedyrt.com/camping/colorado
https://thedyrt.com/camping/connecticut
https://thedyrt.com/camping/delaware
https://thedyrt.com/camping/florida
https://thedyrt.com/camping/georgia
https://thedyrt.com/camping/hawaii
https://thedyrt.com/camping/idaho
https://thedyrt.com/camping/illinois
https://thedyrt.com/camping/indiana
https://thedyrt.com/camping/iowa
https://thedyrt.com/camping/kansas
https://thedyrt.com/camping/kentucky
https://thedyrt.com/camping/louisiana
https://thedyrt.com/camping/maine
https://thedyrt.com/camping/maryland
https://thedyrt.com/camping/massachusetts
https://thedyrt.com/camping/michigan
https://thedyrt.com/camping/minnesota
https://thedyrt.com/camping/mississippi
https://thedyrt.com/camping/missouri
https://thedyrt.com/camping/montana
https://thedyrt.com/

In [3]:

def get_show_more_links(state_pages: list[str]) -> list[str]:

    show_more_urls: list[str] = []

    for url in state_pages:
        resp = requests.get(url)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "lxml")

        btn = soup.find(
            "a",
            attrs={
                "data-event": "seo_page_show_more_campgrounds",
                "data-testid": "seo_page_show_more_campgrounds"
            }
        )
        if btn and btn.get("href"):
            full = urljoin(BASE_URL, btn["href"])
            show_more_urls.append(full)
        else:
            print(f"[!] Show More : {url}")

    return list(dict.fromkeys(show_more_urls))


if __name__ == "__main__":
    links = get_show_more_links(states)
    print(f"[✓] {len(links)} Show More link found:\n")
    for link in links:
        print(link)


[✓] 52 Show More link found:

https://thedyrt.com/search?searchQuery=Alabama&filters=%7B%22region%22%3A%22AL%22%7D&sortBy=recommended
https://thedyrt.com/search?searchQuery=Alaska&filters=%7B%22region%22%3A%22AK%22%7D&sortBy=recommended
https://thedyrt.com/search?searchQuery=Arizona&filters=%7B%22region%22%3A%22AZ%22%7D&sortBy=recommended
https://thedyrt.com/search?searchQuery=Arkansas&filters=%7B%22region%22%3A%22AR%22%7D&sortBy=recommended
https://thedyrt.com/search?searchQuery=California&filters=%7B%22region%22%3A%22CA%22%7D&sortBy=recommended
https://thedyrt.com/search?searchQuery=Colorado&filters=%7B%22region%22%3A%22CO%22%7D&sortBy=recommended
https://thedyrt.com/search?searchQuery=Connecticut&filters=%7B%22region%22%3A%22CT%22%7D&sortBy=recommended
https://thedyrt.com/search?searchQuery=Delaware&filters=%7B%22region%22%3A%22DE%22%7D&sortBy=recommended
https://thedyrt.com/search?searchQuery=Florida&filters=%7B%22region%22%3A%22FL%22%7D&sortBy=recommended
https://thedyrt.com/searc

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

BASE_URL = "https://thedyrt.com"

def init_driver(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless")
        opts.add_argument("--disable-gpu")
    opts.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(options=opts)

def slow_scroll_sidebar(driver, sidebar, pause_time=2, max_attempts=10):
    last_height = driver.execute_script("return arguments[0].scrollHeight;", sidebar)
    attempts = 0

    while attempts < max_attempts:
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", sidebar)
        time.sleep(pause_time)

        new_height = driver.execute_script("return arguments[0].scrollHeight;", sidebar)

        if new_height == last_height:
            attempts += 1
        else:
            attempts = 0

        last_height = new_height

    print("[✓] Scroll completes")

def extract_links_from_search(driver, page_url):
    print(f"\n=== Procces: {page_url}")
    driver.get(page_url)
    wait = WebDriverWait(driver, 20)

    try:
        sidebar = wait.until(EC.presence_of_element_located((By.ID, "sidebar-wrapper")))
        slow_scroll_sidebar(driver, sidebar)
        time.sleep(3)
    except Exception as e:
        print(f"[!] Sidebar cant reload: {page_url}, hata: {e}")
        return []

    anchors = sidebar.find_elements(
        By.CSS_SELECTOR,
        "div.AppCampgroundCard_card--is-campground__qRVuV a.AppCampgroundCard_card__link__RfyjC"
    )

    if not anchors:
        print(f"[!] There is no card: {page_url}")

    card_links = []
    for idx, a in enumerate(anchors, 1):
        href = a.get_attribute("href")
        if href:
            card_links.append(href)
            print(f"[✓] Card #{idx}: {href}")
        else:
            print(f"[!] Card #{idx}: there is no href, HTML:")
            print(a.find_element(By.XPATH, "..").get_attribute("outerHTML")[:300])

    return card_links

def main():
    driver = init_driver()
    all_links = []
    # if you want to get it all states, --> for url in links:
    for url in links[0:1]:
        extracted_links = extract_links_from_search(driver, url)
        all_links.extend(extracted_links)

    driver.quit()

    unique = list(dict.fromkeys(all_links))
    print(f"\n[✓] SUM: {len(unique)} unqiuq links:\n")
    for link in unique:
        print(link)

if __name__ == "__main__":
    main()



=== Procces: https://thedyrt.com/search?searchQuery=Alabama&filters=%7B%22region%22%3A%22AL%22%7D&sortBy=recommended
[✓] Scroll completes
[✓] Card #1: https://thedyrt.com/camping/alabama/alabama-gulf-state-park
[✓] Card #2: https://thedyrt.com/camping/alabama/alabama-oak-mountain-state-park
[✓] Card #3: https://thedyrt.com/camping/alabama/alabama-cheaha-state-park
[✓] Card #4: https://thedyrt.com/camping/alabama/alabama-monte-sano-state-park
[✓] Card #5: https://thedyrt.com/camping/alabama/alabama-desoto-state-park
[✓] Card #6: https://thedyrt.com/camping/alabama/alabama-wind-creek-state-park
[✓] Card #7: https://thedyrt.com/camping/alabama/alabama-lake-guntersville-state-park-campgrounds
[✓] Card #8: https://thedyrt.com/camping/alabama/alabama-gunter-hill
[✓] Card #9: https://thedyrt.com/camping/alabama/noccalula-falls-campground
[✓] Card #10: https://thedyrt.com/camping/alabama/alabama-chewacla-state-park
[✓] Card #11: https://thedyrt.com/camping/alabama/cherokee-rock-village
[✓] Ca

In [ ]:
# We can also retrieve the content of each card link we have obtained using similar scraping methods, or we can send a request by taking the 'slug' information located at the end of the cards.
! curl --location --globoff 'https://thedyrt.com/api/v6/campgrounds?filter[search][slug]=gandy-rv-park' \

{"data":[{"id":"77866","type":"campgrounds","links":{"self":"https://thedyrt.com/api/v6/campgrounds/77866"},"attributes":{"access-types":["drive-in"],"accommodation-types":["rv","tent"],"availability-horizon-end-date":"2026-05-12","camper-types":["backpacker","tent","rv","trailer","other"],"coordinates":{"type":"Point","coordinates":[-87.30339555,31.00361023]},"custom-map":null,"custom-map-data-uri":null,"custom-map-url":null,"latitude":31.00361023,"longitude":-87.30339555,"elevation":257,"price-low":"0.00","price-low-cents":0,"price-low-currency":"USD","price-high":"0.00","price-high-cents":0,"price-high-currency":"USD","number-of-sites":null,"campsites-count":0,"category":"established","description":null,"detailed-category":"rv_park","directions":"IF USING GPS: DO NOT USE LAMBERT RD. Also, PLEASE DO NOT USE GOOGLE MAP DIRECTIONS Please refer to these directions or the map. \r\nFrom Pensacola, FL - From I-10, take HWY 29 North 34 miles to Flomaton, AL, turn left (west) at the first re